In [1]:
import sympy as sp
from sympy import MatrixSymbol, Symbol, symbols
from block_matrix import BlockMatrix, BlockVector, DiagonalBlockMatrix

# Create dimension symbol
n = symbols('n')
inner_dim = n - 1

# Create symbolic matrices with abstract dimension n
m1 = MatrixSymbol('A', inner_dim, inner_dim)  # (n-1)x(n-1) symbolic matrix
m2 = MatrixSymbol('b', inner_dim, 1)          # (n-1)x1 symbolic matrix
m3 = MatrixSymbol('c', 1, inner_dim)          # 1x(n-1) symbolic matrix
m4 = Symbol('d')                              # scalar symbol

# Create block matrix
M = BlockMatrix(m1, m2, m3, m4, n=n)

# Create another symbolic block matrix
p1 = MatrixSymbol('P', inner_dim, inner_dim)
p2 = MatrixSymbol('q', inner_dim, 1)
p3 = MatrixSymbol('r', 1, inner_dim)
p4 = Symbol('s')

N = BlockMatrix(p1, p2, p3, p4, n=n)

# Perform operations
sum_matrix = M + N
product_matrix = M * N
inverse_matrix = M.inverse()

print("Sum of matrices:")
print(sum_matrix)
print("\nProduct of matrices:")
print(product_matrix)
print("\nInverse of first matrix:")
print(inverse_matrix) 

Sum of matrices:
BlockMatrix(
A + P,
b + q,
c + r,
d + s)

Product of matrices:
BlockMatrix(
A*P + b*r,
s*b + A*q,
d*r + c*P,
d*s + Sum(c[0, _i_1]*q[_i_1, 0], (_i_1, 0, n - 2)))

Inverse of first matrix:
BlockMatrix(
A**(-1) + 1/(d - Sum((A**(-1))[_i_1, _i_2]*b[_i_2, 0]*c[0, _i_1], (_i_1, 0, n - 2), (_i_2, 0, n - 2)))*A**(-1)*b*c*A**(-1),
(-1/(d - Sum((A**(-1))[_i_1, _i_2]*b[_i_2, 0]*c[0, _i_1], (_i_1, 0, n - 2), (_i_2, 0, n - 2))))*A**(-1)*b,
(-1/(d - Sum((A**(-1))[_i_1, _i_2]*b[_i_2, 0]*c[0, _i_1], (_i_1, 0, n - 2), (_i_2, 0, n - 2))))*c*A**(-1),
1/(d - Sum((A**(-1))[_i_1, _i_2]*b[_i_2, 0]*c[0, _i_1], (_i_1, 0, n - 2), (_i_2, 0, n - 2))))


In [2]:
v_fix = MatrixSymbol('v', inner_dim, 1)      # (n-1)x1 vector
x = Symbol('x')                              # scalar for last element

# Create block vector
v = BlockVector(v_fix, x, n=n)

# Compute matrix-vector product
result = M * v

print("Block Matrix M:")
print(M)
print("\nBlock Vector v:")
print(v)
print("\nMatrix-vector product M*v:")
print(result)

# Let's also test vector addition and inner product
v2_fix = MatrixSymbol('w', inner_dim, 1)
y = Symbol('y')
v2 = BlockVector(v2_fix, y, n=n)

# Vector addition
sum_vec = v + v2

# Inner product
inner_prod = v.inner_product(v2)

Block Matrix M:
BlockMatrix(
A,
b,
c,
d)

Block Vector v:
BlockVector(
v,
x)

Matrix-vector product M*v:
BlockVector(
x*b + A*v,
d*x + Sum(c[0, _i_1]*v[_i_1, 0], (_i_1, 0, n - 2)))


## Setting up variables for the problem

In [3]:
# Create dimension symbol if not already created
n = symbols('n')
inner_dim = n - 1

# Create components for block matrix K
K1 = MatrixSymbol('K1', inner_dim, inner_dim)  # (n-1)x(n-1) matrix
K2 = MatrixSymbol('K2', inner_dim, 1)          # (n-1)x1 matrix 
K3 = MatrixSymbol('K3', 1, inner_dim)          # 1x(n-1) matrix
K4 = Symbol('K4')                              # scalar

# Create block matrix K
K = BlockMatrix(K1, K2, K3, K4, n=n)

# Create diagonal matrices J and J_1
J_fixed = MatrixSymbol('J_fixed', inner_dim, inner_dim)
j_last = Symbol('j_last')
j1_last = Symbol('j1_last')

J = DiagonalBlockMatrix(J_fixed, j_last, n=n)
J_1 = DiagonalBlockMatrix(J_fixed, j1_last, n=n)

# Create vectors y, y_1 and m, m_1 with shared initial parts
y_fixed = MatrixSymbol('y_fixed', inner_dim, 1)
m_fixed = MatrixSymbol('m_fixed', inner_dim, 1)

y_last = Symbol('y_last')
y1_last = Symbol('y1_last')
m_last = Symbol('m_last')
m1_last = Symbol('m1_last')

y = BlockVector(y_fixed, y_last, n=n)
y_1 = BlockVector(y_fixed, y1_last, n=n)
m = BlockVector(m_fixed, m_last, n=n)
m_1 = BlockVector(m_fixed, m1_last, n=n)